In [1]:
import random

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
print(problem(5,42))

[[0], [1], [0], [4], [0], [1], [4], [4], [4], [1, 3], [0, 1], [2], [1], [0], [0, 2], [2, 4], [3], [3], [4], [2, 4], [0], [1], [0, 1], [3], [2, 3]]


In [4]:
import numpy as np

In [5]:
from gx_utils import *

In [6]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [7]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


In [8]:
import logging
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [9]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [10]:
#return a new_state if elem contains element different from state
def result(state, elem):
    elem_set = set(elem)
    state_set = set()
    for e in state.copy_data():
            [state_set.add(i) for i in e]
    intersection = list(elem_set - state_set)
    if intersection == []: return False

    new_state = []
    [new_state.append(e) for e in state.copy_data().tolist()]
    new_state.append(elem)
    #print(f'append element {elem}')
    new_state = sorted(new_state, key=lambda l: len(l))
    
    return State(np.array(new_state))



In [11]:
#GOAL = State(np.array(range(5)))
GOAL = list(set(range(100)))
logging.info(f"Goal:\n{GOAL}")

def goal_test(state):
    state_set = set()
    test = state.copy_data().tolist()
    for e in test:
        [state_set.add(i) for i in e]
    return list(state_set) == GOAL

INFO:root:Goal:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [18]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    N: int

):  
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0
    state_repeat = 0
    
    GOAL = list(set(range(N)))
    #logging.info(f"Goal:\n{GOAL}")
    
    for elem in input_state:
        while state is not None and not goal_test(state):
            #print(f'n.state {state_repeat}, state: {state}, elem: {elem}')
            #index = input_state.index(elem)
            new_state = result(state, elem)
            # check if new node contains element different from ones in the state
            if not new_state or elem in state.copy_data().tolist():
                break
            else:
                cost = 1
                if new_state not in state_cost and new_state not in frontier:
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    frontier.push(new_state, p=priority_function(new_state))
                    logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
                elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                    old_cost = state_cost[new_state]
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        
            if frontier:
                state = frontier.pop()
            else:
                state = None
        
    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"N = {N} | Found a solution in {len(path):,} steps; visited {len(state_cost):,} states, w = {sum([len(i) for i in state.copy_data().tolist()])}")
    #logging.info(f'statee: {state}')
    #return list(reversed(path))

In [20]:
input_state = sorted(problem(100, seed=42), key=lambda l: len(l))
initial_state = State(np.array([input_state[0]]))
#print(input_state)

# Breadth-first

In [19]:
parent_state = dict()
state_cost = dict()

logging.getLogger().setLevel(logging.INFO)
for n in [5,10,20,50,100,500,1000]:
    input_state = sorted(problem(n, seed=42), key=lambda l: len(l))
    initial_state = State(np.array([input_state[0]]))
    search(
        initial_state=initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost),
        unit_cost=lambda a: 1,
        N=n
    )

INFO:root:N = 5 | Found a solution in 5 steps; visited 5 states, w = 5
C:\Users\peppe\AppData\Local\Temp\ipykernel_12128\4026044864.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return State(np.array(new_state))
INFO:root:N = 10 | Found a solution in 7 steps; visited 7 states, w = 13
INFO:root:N = 20 | Found a solution in 12 steps; visited 12 states, w = 46
INFO:root:N = 50 | Found a solution in 15 steps; visited 15 states, w = 137
INFO:root:N = 100 | Found a solution in 19 steps; visited 19 states, w = 332
INFO:root:N = 500 | Found a solution in 24 steps; visited 24 states, w = 2162
INFO:root:N = 1000 | Found a solution in 26 steps; visited 26 states, w = 4652


# A*

In [15]:
parent_state = dict()
state_cost = dict()


def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))


final = search(
    initial_state=initial_state,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[s] + h(s),
    unit_cost=lambda a: 1,
)

[[32  1 67 68 70  8 76 14 80 49 48 19 20 54 87 59 92]]
n.state 0, state: [[32  1 67 68 70  8 76 14 80 49 48 19 20 54 87 59 92]], elem: [32, 1, 67, 68, 70, 8, 76, 14, 80, 49, 48, 19, 20, 54, 87, 59, 92]
n.state 0, state: [[32  1 67 68 70  8 76 14 80 49 48 19 20 54 87 59 92]], elem: [64, 32, 2, 67, 69, 70, 5, 8, 44, 45, 46, 47, 49, 81, 60, 30, 95]
append element [64, 32, 2, 67, 69, 70, 5, 8, 44, 45, 46, 47, 49, 81, 60, 30, 95]


AttributeError: 'list' object has no attribute '_data'

In [ ]:
l = [1]
m = [1,2,3]

if l in m:
    print('suca')
print('ciao')

prova_state = set(State(np.array(m)).copy_data().tolist())
m = np.array([[3,4],[5]])
intersection = list(set(m) - set(prova_state))

[prova_state.add(i) for i in intersection]
print(prova_state)
print(intersection)


#########################
prova = [5,6,7]
res = result(m, prova)
print(res)

ciao


/var/folders/k_/4h0dvkqj46g0m9nchy6tny8h0000gn/T/ipykernel_8338/3208662567.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m = np.array([[3,4],[5]])


TypeError: unhashable type: 'list'

In [ ]:

l = [1]
m = [[1],[2,3]]
m_1 = State(np.array([1]))
m_23 = State(np.array([2,3]))
m = State(np.array([m_1, m_23]))
#m = State(np.array(m))
#########################
prova = State(np.array([5,6,7]))
new_state = State(np.array([m.copy_data().tolist(), prova.copy_data().tolist()]))
print(State(np.array(new_state)))
m = m.copy_data().tolist()

[list([array([1]), array([2, 3])]) list([5, 6, 7])]


/var/folders/k_/4h0dvkqj46g0m9nchy6tny8h0000gn/T/ipykernel_8338/3867376264.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_state = State(np.array([[i for i in m.copy_data().tolist()], prova.copy_data().tolist()]))


In [ ]:
v_0 = State(np.array([[0], [2,3]]))
v_1 = State(np.array([[0], [2,3]]))

if all(v_0.copy_data() == v_1.copy_data()):
    print('ciao')

arr = [5]

new_state = result(v_0, arr)
print(new_state)

ciao
[0]
[2, 3]
[list([0]) list([2, 3]) list([5])]
[list([0]) list([2, 3]) list([5])]


C:\Users\peppe\AppData\Local\Temp\ipykernel_5856\88107558.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  v_0 = State(np.array([[0], [2,3]]))
C:\Users\peppe\AppData\Local\Temp\ipykernel_5856\88107558.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  v_1 = State(np.array([[0], [2,3]]))
C:\Users\peppe\AppData\Local\Temp\ipykernel_5856\181570025.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to 

In [ ]:
set_arr = set()
for e in v_0.copy_data():
    print(e)
    [set_arr.add(i) for i in e]
print(list(set_arr))

[0]
[2, 3]
[0, 2, 3]


In [ ]:
prova = State(np.array([[0,1],[2,3],[4]]))
prova2 = [2,3]
if prova2 in prova.copy_data().tolist():
    print('ci sono')
print(f'test: {goal_test(prova)}')

ci sono
test: True


C:\Users\peppe\AppData\Local\Temp\ipykernel_4760\3644089678.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prova = State(np.array([[0,1],[2,3],[4]]))
